In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [5]:
ser = Service('../chromedriver_win32/chromedriver.exe')
driver = webdriver.Chrome(service = ser )

In [6]:
url ='https://www.starbucks.co.kr'
driver.get(url)

In [12]:
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, seoul_btn).click()

In [13]:
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, all_btn).click()

In [14]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [15]:
starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))

568


In [18]:
starbucks_soup_list[0:3]

[<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>,
 <li class="quickResultLstCon" data-code="3672" data-hlytag="null" data-index="1" data-lat="37.510178" data-long="127.022223" data-name="논현역사거리" data-storecd="1434" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="논현역사거리" data-store="1434" data-yn="N">논현역사거리  </strong> <p class="result_details">서울특별시 강남구 강남대로 538 (논현동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>,
 <li class="quickResultLstCon" data-code="3858" data-hlytag="null" data-index="2" data-lat="37.514132" data-long="127.020563" data-name="신사역성일빌딩" data-storecd="1595" style="backg

In [33]:
starbucks_store = starbucks_soup_list[0]
name = starbucks_store.select('strong')[0].text.strip()
lat= starbucks_store['data-lat'].strip()
lon=starbucks_store['data-long'].strip()
store_type = starbucks_store.select('i')[0]['class'][0][4:]
address= starbucks_store.select('p.result_details')[0].text[:-9]   # 전화번호가 항목마다 다른 경우엔 다른 방법 필요함.. 
tel = str(starbucks_store.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

print(name)
print(lat)
print(lon)
print(store_type)
print(address)
print(tel)

역삼아레나빌딩
37.501087
127.043069
general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [36]:
starbucks_list = []
columns = ['name', 'latitude', 'longitude', 'store-type', 'address', 'telephone']
for item in starbucks_soup_list:
    name = item.select('strong')[0].text.strip()
    lat = item['data-lat'].strip()
    lon = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0][4:]
    address = item.select('p.result_details')[0].text[:-9]
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    
    starbucks_list.append([name, lat, lon, store_type, address, tel])

- ('tag')  괄호 안에는 태그로 접근. 
- ['attriubute'] 는 starbucks_soup_list에서 출력결과 상의 속성으로 접근. 
- [0] 첫번째 
- split('<') 을 기준으로 나누어[0] 첫번째값. 


In [37]:
columns = ['name', 'latitude', 'longitude', 'store-type', 'address', 'telephone']
seoul_starbucks_df = pd.DataFrame(starbucks_list, columns=columns)
seoul_starbucks_df.head()

,name,latitude,longitude,store-type,address,telephone
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [38]:
seoul_starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        568 non-null    object
 1   latitude    568 non-null    object
 2   longitude   568 non-null    object
 3   store-type  568 non-null    object
 4   address     568 non-null    object
 5   telephone   568 non-null    object
dtypes: object(6)
memory usage: 26.8+ KB


In [40]:
seoul_starbucks_df.to_excel('./seoul_starbucks.xlsx', index = False)

In [41]:
## 
import requests

In [47]:
SEOUL_API_AUTH_KEY = '4244636c58636c613539706d774449'
Open_API = 'GangseoListLoanCompany'

In [48]:
url = 'http://openapi.seoul.go.kr:8088/{}/json/{}/1/5/'.format(SEOUL_API_AUTH_KEY, Open_API)   
# Open API service/start/end 
print(url)

http://openapi.seoul.go.kr:8088/4244636c58636c613539706d774449/json/GangseoListLoanCompany/1/5/


In [49]:
result_dict = requests.get(url).json()
result_dict

{'GangseoListLoanCompany': {'list_total_count': 1104,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'BSNS_SE': '대부중개업',
    'MNG_STATE_CODE': '폐업',
    'OLD_APV_NO': '2015-서울강서-00033(대부중개업)',
    'BUP_NM': '주식회사 론마음파트너스대부중개',
    'REP_NM': '김진영',
    'UPCHE_GBN': '법인',
    'WRKR_BUP_TELNO': '02-521-1012',
    'WRKR_BUP_ADDR': '서울특별시 강서구 등촌동  696번지  세신그린코아빌딩',
    'WRKR_RD_FULL_ADDR': '서울특별시 강서구 공항대로41길 51, 세신그린코아빌딩 9층 905호 (등촌동)',
    'WRKR_BUP_POST_NO': '',
    'APV_YMD': '20210712',
    'END_APV_YMD': '20240712',
    'LAST_CLS_YMD': '20220117',
    'WRKR_ESBMNT_YMD': '20150924',
    'HOFF_YN': '본점',
    'LAST_MOD_YMD': '20220118'},
   {'BSNS_SE': '대부업',
    'MNG_STATE_CODE': '폐업',
    'OLD_APV_NO': '2020-서울강서-0033(대부업)',
    'BUP_NM': '선유대부',
    'REP_NM': '김준영',
    'UPCHE_GBN': '개인',
    'WRKR_BUP_TELNO': '',
    'WRKR_BUP_ADDR': '서울특별시 강서구 가양동  14번지 51호  토피아빌딩',
    'WRKR_RD_FULL_ADDR': '서울특별시 강서구 화곡로 429, 토피아빌딩 4층 402-10호 (가양동)',
    'WRKR_BUP_POST_NO': ''

In [50]:
result_dict['GangseoListLoanCompany']['list_total_count']

1104

In [51]:
result_dict['GangseoListLoanCompany']['RESULT']['CODE']  
# 정상적으로 내용이 뜰 경우: INFO-000 
# 이걸로 try-except 에 걸어줘야 에러 안뜸! 

'INFO-000'

In [52]:
result_dict['GangseoListLoanCompany']['row'][0]['WRKR_BUP_ADDR'].strip()

'서울특별시 강서구 등촌동  696번지  세신그린코아빌딩'

In [55]:
sample_df = pd.DataFrame(result_dict['GangseoListLoanCompany']['row'])
sample_df

,BSNS_SE,MNG_STATE_CODE,OLD_APV_NO,BUP_NM,REP_NM,UPCHE_GBN,WRKR_BUP_TELNO,WRKR_BUP_ADDR,WRKR_RD_FULL_ADDR,WRKR_BUP_POST_NO,APV_YMD,END_APV_YMD,LAST_CLS_YMD,WRKR_ESBMNT_YMD,HOFF_YN,LAST_MOD_YMD
0,대부중개업,폐업,2015-서울강서-00033(대부중개업),주식회사 론마음파트너스대부중개,김진영,법인,02-521-1012,서울특별시 강서구 등촌동 696번지 세신그린코아빌딩,"서울특별시 강서구 공항대로41길 51, 세신그린코아빌딩 9층 905호 (등촌동)",,20210712,20240712,20220117,20150924,본점,20220118
1,대부업,폐업,2020-서울강서-0033(대부업),선유대부,김준영,개인,,서울특별시 강서구 가양동 14번지 51호 토피아빌딩,"서울특별시 강서구 화곡로 429, 토피아빌딩 4층 402-10호 (가양동)",,20201106,20231106,20220117,20201106,본점,20220118
2,대부중개업,영업중,2021-서울강서-0036(대부중개업),EU대부중개,최영우,개인,,서울특별시 강서구 마곡동 798번지 17호,"서울특별시 강서구 마곡중앙1로 10, 8층 801~810호 A31호 (마곡동)",,20210817,20240817,,20210817,본점,20220105
3,대부중개업,폐업,2021-서울강서-0014(대부중개업),와이앤제이컨설팅 대부중개,윤효선,개인,,서울특별시 강서구 마곡동 774번지 1호,"서울특별시 강서구 공항대로 209, 618호 (마곡동)",,20210318,20240318,20220104,20210318,본점,20220104
4,대부업,영업중,2021-서울강서-0053(대부업),꽃길대부,김정근,개인,,서울특별시 강서구 공항동 60번지 118호 대성빌딩,"서울특별시 강서구 송정로 51, 대성빌딩 4층 402-5호 (공항동)",,20211224,20241224,,20211224,본점,20211224


In [57]:
sgg_pop_df = pd.read_csv('./files/report.txt', header = 2, sep = '\t')
#csv는 콤마가 디폴트 seperator 인데 이 자료는 탭으로 구분되어 있어 sep에 \t로 명시해 주었음.  
sgg_pop_df

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2020.3/4,합계,"4,405,833","9,953,009","4,840,912","5,112,097","9,699,232","4,719,170","4,980,062","253,777","121,742","132,035",2.20,"1,552,356"
1,2020.3/4,종로구,"74,861","159,842","77,391","82,451","149,952","73,024","76,928","9,890","4,367","5,523",2.00,"28,396"
2,2020.3/4,중구,"63,594","135,321","66,193","69,128","125,800","61,526","64,274","9,521","4,667","4,854",1.98,"24,265"
3,2020.3/4,용산구,"112,451","244,953","119,074","125,879","229,786","110,604","119,182","15,167","8,470","6,697",2.04,"39,995"
4,2020.3/4,성동구,"136,096","302,695","147,582","155,113","295,591","144,444","151,147","7,104","3,138","3,966",2.17,"45,372"
5,2020.3/4,광진구,"166,857","361,923","174,077","187,846","348,064","168,095","179,969","13,859","5,982","7,877",2.09,"50,047"
6,2020.3/4,동대문구,"167,232","358,679","176,433","182,246","344,416","170,748","173,668","14,263","5,685","8,578",2.06,"61,408"
7,2020.3/4,중랑구,"185,109","400,989","198,012","202,977","395,997","195,981","200,016","4,992","2,031","2,961",2.14,"68,658"
8,2020.3/4,성북구,"195,379","449,871","216,155","233,716","439,719","212,151","227,568","10,152","4,004","6,148",2.25,"73,370"
9,2020.3/4,강북구,"145,790","313,550","152,458","161,092","309,996","151,117","158,879","3,554","1,341","2,213",2.13,"62,951"


In [60]:
SEOUL_API_AUTH_KEY = '4244636c58636c613539706d774449'  
service = 'GangseoListLoanCompany'
url = 'http://openapi.seoul.go.kr:8088/{}/json/{}/1/5/'.format(SEOUL_API_AUTH_KEY, service) 

def seoul_open_api_data(url, service):
    data_list = None
    
    try: 
        result_dict = requests.get(url).json()
        result_data = result_dict[service]
        code = result_data['RESULT']['CODE']
        
        if code == 'INFO-000':    # 정상인지 확인 
            data_list = result_data['row']
            
    except:
        pass 
    return data_list

In [63]:
sgg_pop_df = pd.read_csv('./files/report.txt', header = 2, sep = '\t')
columns = {
    '기간': 'GIGAN',
    '자치구': 'JACHIGU',
    '계': 'GYE_1',
    '계.1': 'GYE_2',
    '계.2': 'GYE_3',
    '남자': 'NAMJA_1',
    '남자.1': 'NAMJA_2',
    '남자.2': 'NAMJA_3',
    '여자': 'YEOJA_1',
    '여자.1': 'YEOJA_2',
    '여자.2': 'YEOJA_3',
    '세대': 'SEDAE',
    '세대당인구': 'SEDAEDANGINGU',
    '65세이상고령자': 'N_65SEISANGGORYEONGJA'
}
sgg_pop_df.rename(columns = columns, inplace = True)
sgg_pop_df

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
0,2020.3/4,합계,"4,405,833","9,953,009","4,840,912","5,112,097","9,699,232","4,719,170","4,980,062","253,777","121,742","132,035",2.20,"1,552,356"
1,2020.3/4,종로구,"74,861","159,842","77,391","82,451","149,952","73,024","76,928","9,890","4,367","5,523",2.00,"28,396"
2,2020.3/4,중구,"63,594","135,321","66,193","69,128","125,800","61,526","64,274","9,521","4,667","4,854",1.98,"24,265"
3,2020.3/4,용산구,"112,451","244,953","119,074","125,879","229,786","110,604","119,182","15,167","8,470","6,697",2.04,"39,995"
4,2020.3/4,성동구,"136,096","302,695","147,582","155,113","295,591","144,444","151,147","7,104","3,138","3,966",2.17,"45,372"
5,2020.3/4,광진구,"166,857","361,923","174,077","187,846","348,064","168,095","179,969","13,859","5,982","7,877",2.09,"50,047"
6,2020.3/4,동대문구,"167,232","358,679","176,433","182,246","344,416","170,748","173,668","14,263","5,685","8,578",2.06,"61,408"
7,2020.3/4,중랑구,"185,109","400,989","198,012","202,977","395,997","195,981","200,016","4,992","2,031","2,961",2.14,"68,658"
8,2020.3/4,성북구,"195,379","449,871","216,155","233,716","439,719","212,151","227,568","10,152","4,004","6,148",2.25,"73,370"
9,2020.3/4,강북구,"145,790","313,550","152,458","161,092","309,996","151,117","158,879","3,554","1,341","2,213",2.13,"62,951"


In [64]:
sgg_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GIGAN                  26 non-null     object 
 1   JACHIGU                26 non-null     object 
 2   SEDAE                  26 non-null     object 
 3   GYE_1                  26 non-null     object 
 4   NAMJA_1                26 non-null     object 
 5   YEOJA_1                26 non-null     object 
 6   GYE_2                  26 non-null     object 
 7   NAMJA_2                26 non-null     object 
 8   YEOJA_2                26 non-null     object 
 9   GYE_3                  26 non-null     object 
 10  NAMJA_3                26 non-null     object 
 11  YEOJA_3                26 non-null     object 
 12  SEDAEDANGINGU          26 non-null     float64
 13  N_65SEISANGGORYEONGJA  26 non-null     object 
dtypes: float64(1), object(13)
memory usage: 3.0+ KB


In [65]:
condition = sgg_pop_df['JACHIGU'] != '합계'
condition

0     False
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
Name: JACHIGU, dtype: bool

In [67]:
sgg_pop_df_select = sgg_pop_df[condition]   # sgg_pop_df_select은 첫줄의 합계가 지워진 데이터

In [68]:
sgg_pop_df_select.head()

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
1,2020.3/4,종로구,"74,861","159,842","77,391","82,451","149,952","73,024","76,928","9,890","4,367","5,523",2.00,"28,396"
2,2020.3/4,중구,"63,594","135,321","66,193","69,128","125,800","61,526","64,274","9,521","4,667","4,854",1.98,"24,265"
3,2020.3/4,용산구,"112,451","244,953","119,074","125,879","229,786","110,604","119,182","15,167","8,470","6,697",2.04,"39,995"
4,2020.3/4,성동구,"136,096","302,695","147,582","155,113","295,591","144,444","151,147","7,104","3,138","3,966",2.17,"45,372"
5,2020.3/4,광진구,"166,857","361,923","174,077","187,846","348,064","168,095","179,969","13,859","5,982","7,877",2.09,"50,047"


In [70]:
sgg_pop_df_final = sgg_pop_df_select[['JACHIGU', 'GYE_1']]
sgg_pop_df_final

,JACHIGU,GYE_1
1,종로구,"159,842"
2,중구,"135,321"
3,용산구,"244,953"
4,성동구,"302,695"
5,광진구,"361,923"
6,동대문구,"358,679"
7,중랑구,"400,989"
8,성북구,"449,871"
9,강북구,"313,550"
10,도봉구,"329,300"


In [71]:
sgg_pop_df_final.columns = ['시군구명', '주민등록인구']
sgg_pop_df_final.to_excel('./sgg_pop.xlsx', index=False)

In [73]:
sgg_biz_df = pd.read_csv('./files/report2.txt', sep = '\t', header = 2)
columns = {
    '기간': 'GIGAN',
    '자치구': 'JACHIGU',
    '동': 'DONG',
    '사업체수': 'SAEOPCHESU_1',
    '여성대표자': 'YEOSEONGDAEPYOJA',
    '계': 'GYE',
    '남': 'NAM',
    '여': 'YEO'
}

sgg_biz_df.rename(columns=columns, inplace=True)
sgg_biz_df.head(10)

,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,사업체수.1,종사자수,...,사업체수.15,종사자수.14,사업체수.16,종사자수.15,사업체수.17,종사자수.16,사업체수.18,종사자수.17,사업체수.19,종사자수.18
0,2018,합계,합계,"823,385","279,711","5,210,936","2,878,227","2,332,709",30,462,...,"1,282","140,121","35,377","346,219","28,824","378,366","23,025","91,104","69,694","191,423"
1,2018,종로구,소계,"39,952","13,442","265,017","152,717","112,300",3,17,...,89,"15,658",850,"13,504",744,"15,888",909,"6,227","2,448","7,672"
2,2018,종로구,사직동,"3,541","1,191","49,536","28,658","20,878",1,14,...,19,"6,312",86,967,103,870,95,"1,544",252,843
3,2018,종로구,삼청동,712,324,"4,577","2,193","2,384",-,-,...,4,"1,204",14,260,7,30,17,175,24,209
4,2018,종로구,부암동,565,240,"3,609","1,706","1,903",-,-,...,3,59,40,"1,424",15,148,16,40,83,362
5,2018,종로구,평창동,751,319,"3,565","1,670","1,895",-,-,...,3,85,56,618,36,466,32,186,115,262
6,2018,종로구,무악동,581,103,"1,598",803,795,-,-,...,1,16,25,185,11,482,4,12,44,102
7,2018,종로구,교남동,393,156,"4,008","2,486","1,522",-,-,...,2,40,18,176,21,"2,499",11,44,57,181
8,2018,종로구,가회동,766,372,"14,100","10,618","3,482",-,-,...,3,459,32,691,14,83,37,151,58,249
9,2018,종로구,종로1.2.3.4가동,"15,603","4,989","109,824","66,387","43,437",-,-,...,23,"4,886",301,"3,168",200,"1,568",249,"1,381",884,"2,757"


In [74]:
condition = sgg_biz_df['DONG'] =='소계'
sgg_biz_df_selected = sgg_biz_df[condition]

sgg_biz_df_selected.head(10)

,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,사업체수.1,종사자수,...,사업체수.15,종사자수.14,사업체수.16,종사자수.15,사업체수.17,종사자수.16,사업체수.18,종사자수.17,사업체수.19,종사자수.18
1,2018,종로구,소계,"39,952","13,442","265,017","152,717","112,300",3,17,...,89,"15,658",850,"13,504",744,"15,888",909,"6,227","2,448","7,672"
19,2018,중구,소계,"60,957","22,669","390,530","210,755","179,775",-,-,...,62,"19,280",471,"8,321",747,"10,104",558,"3,723","2,138","8,366"
35,2018,용산구,소계,"20,813","7,153","141,216","76,498","64,718",-,-,...,47,"3,149",550,"7,452",579,"6,658",408,"2,831","1,641","5,939"
52,2018,성동구,소계,"27,868","8,432","174,819","100,694","74,125",1,3,...,47,"4,471",961,"11,371",748,"8,539",581,"2,157","2,115","7,612"
70,2018,광진구,소계,"24,535","8,948","127,879","69,442","58,437",1,3,...,36,"2,662","1,183","14,139",909,"11,615","1,033","3,862","2,587","5,608"
86,2018,동대문구,소계,"31,878","10,575","140,748","74,303","66,445",-,-,...,50,"3,443","1,000","16,243","1,031","14,494",776,"2,462","2,581","5,466"
101,2018,중랑구,소계,"28,024","9,410","103,016","53,541","49,475",-,-,...,47,"3,471",936,"7,773",999,"11,920",940,"2,484","2,739","5,153"
118,2018,성북구,소계,"23,287","8,831","108,201","52,719","55,482",-,-,...,52,"3,301","1,382","21,481","1,044","12,835",756,"2,306","2,684","5,722"
139,2018,강북구,소계,"18,918","7,065","72,418","33,306","39,112",-,-,...,44,"3,265",684,"5,871",813,"9,195",741,"2,037","2,285","4,503"
153,2018,도봉구,소계,"18,894","6,320","74,314","38,646","35,668",1,3,...,38,"3,872",962,"7,537",837,"11,640",622,"1,907","2,064","4,366"


In [75]:
columns = ['JACHIGU', 'GYE', 'SAEOPCHESU_1']
sgg_biz_df_final = sgg_biz_df_selected[columns]
sgg_biz_df_final.columns = ['시군구명', '종사자수', '사업체수']
sgg_biz_df_final.head()

,시군구명,종사자수,사업체수
1,종로구,"265,017","39,952"
19,중구,"390,530","60,957"
35,용산구,"141,216","20,813"
52,성동구,"174,819","27,868"
70,광진구,"127,879","24,535"


In [76]:
sgg_biz_df_final = sgg_biz_df_final.reset_index(drop = True)
sgg_biz_df_final.head()

,시군구명,종사자수,사업체수
0,종로구,"265,017","39,952"
1,중구,"390,530","60,957"
2,용산구,"141,216","20,813"
3,성동구,"174,819","27,868"
4,광진구,"127,879","24,535"


In [77]:
sgg_biz_df_final.to_excel('./sgg_biz.xlsx', index=False)

### 스타벅스 지점이 어떤 구에 있는지 확인하기. 

In [78]:
seoul_starbucks = pd.read_excel('./seoul_starbucks.xlsx', header = 0)
seoul_starbucks.head()

,name,latitude,longitude,store-type,address,telephone
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [82]:
sgg_names = []
for address in seoul_starbucks['address']:
    sgg = address.split()[1]    # 공백기준 쪼갠다음 두번째 것
    sgg_names.append(sgg)
    
seoul_starbucks['시군구명'] = sgg_names
seoul_starbucks.head(100)

,name,latitude,longitude,store-type,address,telephone,시군구명
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232,강남구
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,강남구
4,대치재경빌딩R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,강남구
...,...,...,...,...,...,...,...
95,마곡역,37.559611,126.826298,general,서울특별시 강서구 공항대로 168 (마곡동),1522-3232,강서구
96,우장산역,37.549501,126.835984,general,서울특별시 강서구 강서로 267 (내발산동),1522-3232,강서구
97,마곡사이언스타워R,37.560526,126.833887,reserve,서울특별시 강서구 마곡동로 62 (마곡동),1522-3232,강서구
98,방화DT,37.574339,126.816415,generalDT,서울특별시 강서구 방화대로 375 (방화동),1522-3232,강서구


### 시군구 목록 불러서 병합 

In [84]:
seoul_sgg = pd.read_excel('./files/seoul_sgg_list.xlsx')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.665861,127.031767
1,11380,은평구,37.617612,126.922700
2,11230,동대문구,37.583801,127.050700
3,11590,동작구,37.496504,126.944307
4,11545,금천구,37.460097,126.900155


In [87]:
#시군구별 매장 수 count 
starbucks_sgg_count = seoul_starbucks.pivot_table(index='시군구명', values='name', aggfunc='count').rename(columns={'name':'스타벅스_매장수' })
starbucks_sgg_count.head()

,스타벅스_매장수
시군구명,
강남구,88
강동구,16
강북구,6
강서구,22
관악구,11
